In [1]:
import torch
from torch import nn

step =3

a = torch.linspace(-1., 1., steps=step, dtype=torch.float).view(1, step).expand(step, step).reshape(1, -1)
b = torch.linspace(-1., 1., steps=step, dtype=torch.float).view(step, 1).expand(step, step).reshape(1, -1)
c= torch.cat([a, b], dim=0).cuda()

print(a)
print(a.shape)

tensor([[-1.,  0.,  1., -1.,  0.,  1., -1.,  0.,  1.]])
torch.Size([1, 9])


In [2]:
print(b)
print(b.shape)

tensor([[-1., -1., -1.,  0.,  0.,  0.,  1.,  1.,  1.]])
torch.Size([1, 9])


In [3]:
print(c)
print(c.shape)

tensor([[-1.,  0.,  1., -1.,  0.,  1., -1.,  0.,  1.],
        [-1., -1., -1.,  0.,  0.,  0.,  1.,  1.,  1.]], device='cuda:0')
torch.Size([2, 9])


In [4]:
model = nn.Conv1d(2, 3, 2)
c = torch.randn(2,9)

print(model(c).shape)

torch.Size([3, 8])


In [5]:
in_channel = 6
hidden_dim = 20

a = nn.Sequential(
            nn.Conv1d(in_channel + 2, hidden_dim, 1),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(inplace=True),
            nn.Conv1d(hidden_dim, hidden_dim//2, 1),
            nn.BatchNorm1d(hidden_dim//2),
            nn.ReLU(inplace=True),
            nn.Conv1d(hidden_dim//2, 3, 1),
        )

In [6]:
c = torch.randn(44,8,20)
v = a(c)
print(v.shape)

torch.Size([44, 3, 20])


In [7]:
# Qua PoinTr
trans_dim = 384
knn_layer = 1
num_pred = 14336
num_query = 224

fold_step = int(pow(num_pred//num_query, 0.5) + 0.5)
print("fold_step: ", fold_step)

fold_step:  8


In [8]:
a = torch.randn(2,3,4)
a = a.transpose(-1,-2)
print(a.shape)

torch.Size([2, 4, 3])


In [9]:
bs = 10
c = 3
n = 20

In [10]:
coor_q = torch.randn(bs,c,n)
x = torch.randn(bs,8,n)
model = nn.Conv1d(3,8,1)
b = model(coor_q)
print(b.shape)

torch.Size([10, 8, 20])


In [11]:
def knn_point(nsample, xyz, new_xyz):
    """
    Input:
        nsample: max sample number in local region     k =  16
        xyz: all points, [B, N, C]      B, N, 3
        new_xyz: query points, [B, S, C]         B, N, 3
    Return:
        group_idx: grouped points index, [B, S, nsample]
    """
    sqrdists = square_distance(new_xyz, xyz)
    print("sqrdists.shape",sqrdists.shape)
    _, group_idx = torch.topk(sqrdists, nsample, dim = -1, largest=False, sorted=False)
    return group_idx

def square_distance(src, dst):
    """
    Calculate Euclid distance between each two points.
    src^T * dst = xn * xm + yn * ym + zn * zm
    sum(src^2, dim=-1) = xn*xn + yn*yn + zn*zn;
    sum(dst^2, dim=-1) = xm*xm + ym*ym + zm*zm;
    dist = (xn - xm)^2 + (yn - ym)^2 + (zn - zm)^2
         = sum(src**2,dim=-1)+sum(dst**2,dim=-1)-2*src^T*dst
    Input:
        src: source points, [B, N, C]
        dst: target points, [B, M, C]
    Output:
        dist: per-point square distance, [B, N, M]
    """
    B, N, _ = src.shape
    _, M, _ = dst.shape
    dist = -2 * torch.matmul(src, dst.permute(0, 2, 1))
    dist += torch.sum(src ** 2, -1).view(B, N, 1)
    dist += torch.sum(dst ** 2, -1).view(B, 1, M)
    return dist

In [12]:
idx = knn_point(16, coor_q.transpose(-1,-2).contiguous(),coor_q.transpose(-1,-2).contiguous())
idx = idx.transpose(-1,-2).contiguous()
print(idx.shape)

# from C = 10,30,3 -------> 10,30,16

sqrdists.shape torch.Size([10, 20, 20])
torch.Size([10, 16, 20])


In [13]:
print(idx[0])

tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19],
        [11,  5, 19,  8, 18,  1,  1, 19,  3, 13, 13,  0,  3, 10,  5,  8,  1, 13,
          4, 14],
        [ 2, 14, 13, 19, 16, 14,  5, 14, 11, 10,  9,  2,  6, 17,  1,  3,  5, 10,
         10,  2],
        [17, 16,  5, 11, 14, 19, 16,  3,  0, 16, 17,  3,  2,  9, 19, 11,  9,  9,
         16,  3],
        [ 3, 19, 11,  2,  5, 16,  9,  5, 19,  2, 18, 19, 19,  2,  7,  7, 14,  2,
         13,  7],
        [19,  2,  1,  7,  1,  2,  2, 11,  7, 17,  2,  8,  8, 19,  2,  0, 18,  0,
          9,  5],
        [ 8,  6,  0, 14, 10,  7, 12,  2,  2,  1, 16,  7,  1, 11, 16, 19, 10, 11,
         14,  1],
        [13,  9,  9,  0,  7,  6, 14, 13, 12,  5, 19, 13,  5,  7,  3,  2, 13, 19,
          7, 11],
        [ 7,  7, 17, 12, 13,  9, 19,  1, 15,  6,  5, 17,  0,  5, 13, 14,  6,  1,
          5, 13],
        [10, 13, 14,  5,  9, 13, 13, 10, 14, 19,  7, 14, 11, 16,  9, 12,  2,  5,
          1,  0],
        [1

In [14]:
idx_base = torch.arange(0, bs).view(-1,1,1) * n
print(idx_base)


tensor([[[  0]],

        [[ 20]],

        [[ 40]],

        [[ 60]],

        [[ 80]],

        [[100]],

        [[120]],

        [[140]],

        [[160]],

        [[180]]])


In [15]:
print(idx[0])

tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19],
        [11,  5, 19,  8, 18,  1,  1, 19,  3, 13, 13,  0,  3, 10,  5,  8,  1, 13,
          4, 14],
        [ 2, 14, 13, 19, 16, 14,  5, 14, 11, 10,  9,  2,  6, 17,  1,  3,  5, 10,
         10,  2],
        [17, 16,  5, 11, 14, 19, 16,  3,  0, 16, 17,  3,  2,  9, 19, 11,  9,  9,
         16,  3],
        [ 3, 19, 11,  2,  5, 16,  9,  5, 19,  2, 18, 19, 19,  2,  7,  7, 14,  2,
         13,  7],
        [19,  2,  1,  7,  1,  2,  2, 11,  7, 17,  2,  8,  8, 19,  2,  0, 18,  0,
          9,  5],
        [ 8,  6,  0, 14, 10,  7, 12,  2,  2,  1, 16,  7,  1, 11, 16, 19, 10, 11,
         14,  1],
        [13,  9,  9,  0,  7,  6, 14, 13, 12,  5, 19, 13,  5,  7,  3,  2, 13, 19,
          7, 11],
        [ 7,  7, 17, 12, 13,  9, 19,  1, 15,  6,  5, 17,  0,  5, 13, 14,  6,  1,
          5, 13],
        [10, 13, 14,  5,  9, 13, 13, 10, 14, 19,  7, 14, 11, 16,  9, 12,  2,  5,
          1,  0],
        [1

In [16]:
idx = idx + idx_base
print(idx.shape)
print(idx[0])

torch.Size([10, 16, 20])
tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19],
        [11,  5, 19,  8, 18,  1,  1, 19,  3, 13, 13,  0,  3, 10,  5,  8,  1, 13,
          4, 14],
        [ 2, 14, 13, 19, 16, 14,  5, 14, 11, 10,  9,  2,  6, 17,  1,  3,  5, 10,
         10,  2],
        [17, 16,  5, 11, 14, 19, 16,  3,  0, 16, 17,  3,  2,  9, 19, 11,  9,  9,
         16,  3],
        [ 3, 19, 11,  2,  5, 16,  9,  5, 19,  2, 18, 19, 19,  2,  7,  7, 14,  2,
         13,  7],
        [19,  2,  1,  7,  1,  2,  2, 11,  7, 17,  2,  8,  8, 19,  2,  0, 18,  0,
          9,  5],
        [ 8,  6,  0, 14, 10,  7, 12,  2,  2,  1, 16,  7,  1, 11, 16, 19, 10, 11,
         14,  1],
        [13,  9,  9,  0,  7,  6, 14, 13, 12,  5, 19, 13,  5,  7,  3,  2, 13, 19,
          7, 11],
        [ 7,  7, 17, 12, 13,  9, 19,  1, 15,  6,  5, 17,  0,  5, 13, 14,  6,  1,
          5, 13],
        [10, 13, 14,  5,  9, 13, 13, 10, 14, 19,  7, 14, 11, 16,  9, 12,  2,  5,
   

In [17]:
print(idx[1])

tensor([[20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37,
         38, 39],
        [24, 29, 28, 36, 20, 30, 28, 35, 22, 21, 36, 37, 28, 28, 20, 36, 30, 31,
         25, 24],
        [25, 39, 31, 30, 38, 38, 32, 28, 32, 33, 35, 22, 26, 29, 37, 30, 35, 22,
         35, 25],
        [30, 24, 24, 25, 22, 36, 22, 36, 26, 32, 25, 24, 22, 38, 24, 38, 25, 20,
         24, 20],
        [39, 38, 37, 34, 25, 20, 27, 22, 33, 24, 38, 20, 31, 35, 23, 25, 38, 24,
         30, 38],
        [38, 33, 20, 35, 39, 35, 31, 26, 27, 28, 20, 32, 29, 32, 30, 27, 23, 34,
         36, 21],
        [34, 20, 32, 20, 30, 24, 37, 30, 24, 39, 24, 28, 33, 27, 25, 24, 24, 30,
         20, 30],
        [22, 25, 38, 38, 31, 39, 33, 38, 38, 38, 22, 29, 24, 22, 31, 22, 27, 32,
         39, 29],
        [37, 31, 30, 24, 37, 23, 24, 33, 31, 31, 23, 38, 37, 24, 39, 20, 20, 28,
         22, 35],
        [31, 32, 26, 37, 35, 22, 35, 24, 29, 22, 27, 26, 38, 25, 22, 28, 22, 26,
         33, 22],
        [3

In [18]:
idx = idx.view(-1)
print(idx.shape)
print(idx[1000:1102])
#assert idx == a

torch.Size([3200])
tensor([62, 71, 77, 77, 76, 74, 68, 77, 66, 67, 65, 61, 75, 61, 60, 67, 68, 67,
        65, 61, 65, 68, 72, 66, 79, 70, 63, 69, 76, 72, 64, 68, 69, 63, 65, 72,
        63, 62, 64, 71, 75, 79, 63, 76, 66, 76, 65, 72, 73, 60, 60, 63, 62, 68,
        62, 60, 65, 68, 74, 65, 69, 66, 68, 73, 70, 60, 64, 63, 77, 71, 78, 69,
        77, 67, 64, 71, 64, 72, 76, 76, 68, 63, 74, 67, 74, 66, 71, 73, 71, 73,
        62, 75, 60, 66, 76, 73, 71, 66, 66, 66, 71, 76])


In [19]:
#x = torch.rand(10,30,8)
feature = x.view(bs*n,-1)
print(feature)
print(feature.shape)
#features = feature[idx, :]
#

tensor([[ 0.8655,  0.0401,  0.7722,  ...,  1.1129,  0.1859, -1.5990],
        [-0.5356,  0.0232, -0.0681,  ...,  0.9321,  1.4021,  0.6932],
        [ 0.2074,  0.5358,  0.2506,  ...,  1.2926,  0.7123,  0.2106],
        ...,
        [ 1.3767,  0.3430, -0.0890,  ..., -0.4585, -0.0672,  0.4633],
        [-0.7804, -2.1200, -1.1222,  ..., -0.5658, -0.8020, -1.3861],
        [-0.0228,  1.5791, -0.6177,  ...,  0.7621, -2.1129,  1.0138]])
torch.Size([200, 8])


In [20]:
feature = feature[idx, :]
print(feature)
print(feature.shape)

tensor([[ 8.6551e-01,  4.0063e-02,  7.7224e-01,  ...,  1.1129e+00,
          1.8590e-01, -1.5990e+00],
        [-5.3556e-01,  2.3208e-02, -6.8061e-02,  ...,  9.3206e-01,
          1.4021e+00,  6.9316e-01],
        [ 2.0738e-01,  5.3582e-01,  2.5064e-01,  ...,  1.2926e+00,
          7.1225e-01,  2.1060e-01],
        ...,
        [ 1.3414e+00, -1.2810e+00, -4.6848e-01,  ...,  9.5121e-01,
          1.1494e-01,  2.2980e-01],
        [-2.2842e-02,  1.5791e+00, -6.1775e-01,  ...,  7.6211e-01,
         -2.1129e+00,  1.0138e+00],
        [ 7.5595e-01,  1.1761e+00,  9.0798e-01,  ...,  1.3700e+00,
          5.3536e-04,  1.3930e+00]])
torch.Size([3200, 8])


In [21]:
feature = feature.view(bs, 16, n, 8).permute(0, 3, 2, 1).contiguous()


In [22]:
print(feature.shape)
#FROM  3200,8 ---> 10,8,20,16 is bs,c,n,k
x = x.view(bs, 8, n, 1).expand(-1, -1, -1, 16)

torch.Size([10, 8, 20, 16])


In [23]:
print(x[0,0,0])

tensor([0.8655, 0.8655, 0.8655, 0.8655, 0.8655, 0.8655, 0.8655, 0.8655, 0.8655,
        0.8655, 0.8655, 0.8655, 0.8655, 0.8655, 0.8655, 0.8655])


In [24]:
feature = torch.cat((feature - x, x), dim=1)

In [25]:
print(feature.shape)

torch.Size([10, 16, 20, 16])


In [26]:
# groupnorm = nn.GroupNorm(2,4)
# test = torch.tensor([[1,2,3,4],[1,9,5,4],[1,4,1,4],[1,8,7,4],[1,4,1,1]],dtype=torch.float)  # 5xC(4)
# result = groupnorm(test)
# print(result)
layer1 = nn.Sequential(nn.Conv2d(16, 32, kernel_size=1, bias=False), # B,8,N,16 --> B,32,N,16
                                   nn.GroupNorm(4, 32),
                                   nn.LeakyReLU(negative_slope=0.2)
                                   )

In [27]:
test = torch.tensor([[1,2,3,4],[1,9,5,4],[1,4,1,4],[1,8,7,4],[1,4,1,1]],dtype=torch.float)  # 5xC(4)
t = test.max(-1,keepdim=False)
print(t)

torch.return_types.max(
values=tensor([4., 9., 4., 8., 4.]),
indices=tensor([3, 1, 1, 1, 1]))


In [28]:
f = layer1(feature)
print(f.shape)
f = f.max(-1, keepdim=False)
print(f)

torch.Size([10, 32, 20, 16])
torch.return_types.max(
values=tensor([[[ 2.8481,  3.0120,  2.1794,  ...,  0.4912,  1.4207,  1.5490],
         [-0.0667,  0.3170,  1.2922,  ...,  0.0425,  0.9726,  0.7651],
         [ 0.5465,  1.5642,  0.7405,  ..., -0.1106, -0.1196,  0.8093],
         ...,
         [ 2.1218,  1.0358,  0.4066,  ...,  2.3124,  1.3226,  1.2843],
         [ 1.6370,  0.7463,  0.4448,  ...,  1.0949,  0.9119, -0.1012],
         [-0.0636, -0.1535,  0.6616,  ...,  1.5665,  2.4998,  0.8026]],

        [[ 2.1225,  2.2735,  0.4287,  ...,  0.3421,  1.5130,  0.1804],
         [ 2.9372,  1.4094,  0.7583,  ...,  0.2786, -0.2942,  1.4066],
         [ 1.0572,  0.9606,  0.8761,  ...,  0.5188,  0.8294,  1.5219],
         ...,
         [-0.0200,  0.4338,  2.3987,  ...,  2.7123,  2.3416,  0.6139],
         [-0.1036,  0.3965, -0.0436,  ..., -0.0293,  2.6460,  0.7539],
         [ 2.3868,  0.7581,  1.4240,  ...,  1.5070, -0.0650,  1.6863]],

        [[ 1.2612,  0.2019,  0.0183,  ...,  1.4158, -0.0

In [29]:
a  = torch.randn(3,20,3)
print(a.shape)


torch.Size([3, 20, 3])


In [30]:
center = torch.mean(a, dim=1)
print(center)

tensor([[-0.1500,  0.0311, -0.1534],
        [-0.4396,  0.3668,  0.1480],
        [ 0.1105,  0.6169, -0.0097]])


In [31]:
c = torch.tensor([[1,2,3,4,5],[1,2,1,2,1],[2,3,2,3,3]])
f = torch.tensor([[1,2,1,2,2]])
h = (c-f)*(c-f)
print(h)
print(c.shape,f.shape)
f = f.reshape(5,1,1)
print(f.shape)

tensor([[0, 0, 4, 4, 9],
        [0, 0, 0, 0, 1],
        [1, 1, 1, 1, 1]])
torch.Size([3, 5]) torch.Size([1, 5])
torch.Size([5, 1, 1])


In [32]:
dd = torch.concat([c,h],dim =1)
print(dd.shape)

torch.Size([3, 10])


In [33]:
g = dd*dd
print(g.shape)

torch.Size([3, 10])


In [34]:
v = torch.sum(g,dim=1)
print(v)

tensor([168,  12,  40])


In [35]:
vv = torch.sort(g,dim=1)
print(vv)
print(g)

torch.return_types.sort(
values=tensor([[ 0,  0,  1,  4,  9, 16, 16, 16, 25, 81],
        [ 0,  0,  0,  0,  1,  1,  1,  1,  4,  4],
        [ 1,  1,  1,  1,  1,  4,  4,  9,  9,  9]]),
indices=tensor([[5, 6, 0, 1, 2, 3, 7, 8, 4, 9],
        [5, 6, 7, 8, 0, 2, 4, 9, 1, 3],
        [5, 6, 7, 8, 9, 0, 2, 1, 3, 4]]))
tensor([[ 1,  4,  9, 16, 25,  0,  0, 16, 16, 81],
        [ 1,  4,  1,  4,  1,  0,  0,  0,  0,  1],
        [ 4,  9,  4,  9,  9,  1,  1,  1,  1,  1]])


In [36]:
vip = torch.randn(2,3,1)
print(vip)


tensor([[[ 2.3033],
         [-0.8871],
         [ 0.6483]],

        [[ 0.0495],
         [-0.0875],
         [-1.6272]]])


In [37]:
sortvip = torch.sort(vip,dim12)
print(sortvip)

NameError: name 'dim12' is not defined

In [38]:
a = torch.rand(5, 10, 1)  # Replace with your actual tensor "a"
_, index_sort = torch.sort(a, dim=1)  # Get sorting indices for "a" on dim=1 (10 here)

b = torch.rand(5, 10, 3)  # Replace with your actual tensor "b"
print(index_sort.shape)


torch.Size([5, 10, 1])


In [39]:
expanded_indices = index_sort[:, :].expand_as(b)
print(expanded_indices.shape)

torch.Size([5, 10, 3])


In [40]:
sorted_b = torch.gather(b, dim=1, index=expanded_indices)


In [41]:
print(sorted_b.shape)
print(sorted_b[0])
print(b[0])
print(expanded_indices)

torch.Size([5, 10, 3])
tensor([[0.8331, 0.9963, 0.2219],
        [0.6938, 0.5713, 0.2754],
        [0.7270, 0.9883, 0.1749],
        [0.3538, 0.6450, 0.0524],
        [0.6971, 0.2312, 0.1324],
        [0.8141, 0.8299, 0.9632],
        [0.1308, 0.1690, 0.1791],
        [0.7684, 0.2833, 0.1349],
        [0.8713, 0.2591, 0.6996],
        [0.4581, 0.8400, 0.2452]])
tensor([[0.8141, 0.8299, 0.9632],
        [0.3538, 0.6450, 0.0524],
        [0.7270, 0.9883, 0.1749],
        [0.6938, 0.5713, 0.2754],
        [0.1308, 0.1690, 0.1791],
        [0.8713, 0.2591, 0.6996],
        [0.7684, 0.2833, 0.1349],
        [0.4581, 0.8400, 0.2452],
        [0.6971, 0.2312, 0.1324],
        [0.8331, 0.9963, 0.2219]])
tensor([[[9, 9, 9],
         [3, 3, 3],
         [2, 2, 2],
         [1, 1, 1],
         [8, 8, 8],
         [0, 0, 0],
         [4, 4, 4],
         [6, 6, 6],
         [5, 5, 5],
         [7, 7, 7]],

        [[6, 6, 6],
         [7, 7, 7],
         [9, 9, 9],
         [4, 4, 4],
         [0, 

In [42]:
cv = torch.randn(3,2)
vc = cv.view(3,-1,1)
print(vc.shape)
print(cv)
print(vc)

torch.Size([3, 2, 1])
tensor([[ 0.8291,  1.4782],
        [-1.6637, -1.2961],
        [-1.0994,  0.3817]])
tensor([[[ 0.8291],
         [ 1.4782]],

        [[-1.6637],
         [-1.2961]],

        [[-1.0994],
         [ 0.3817]]])


In [43]:
coor = torch.randn(3,10,3)
f = torch.randn(3,10,4)
bs = 3

In [44]:
center_coor = torch.mean(coor,dim=1) # dim = bs * 3
center_coor = center_coor.reshape(bs,1,3)
dis_to_center = coor - center_coor # bs * 256 * 3
dis_to_center = torch.sum(dis_to_center*dis_to_center,dim=2) # bs* 256 *1
dis_to_center = dis_to_center.view(bs,-1,1)
dis_to_center,sorted_dis_index = torch.sort(dis_to_center, dim =1) # bs * 256 * 1
expanded_indices = sorted_dis_index[:, :].expand_as(coor)
coor = torch.gather(coor, dim=1, index=expanded_indices)  # sort coor as dis_to_center
expanded_indices_f = sorted_dis_index[:, :].expand_as(f)
f = torch.gather(f, dim=1, index=expanded_indices_f)  # sort f as dis_to_center

In [49]:
print(dis_to_center.shape)
alo = torch.nn.functional.normalize(dis_to_center*dis_to_center,dim=1)
print(alo.shape,alo)

torch.Size([3, 10, 1])
torch.Size([3, 10, 1]) tensor([[[0.0127],
         [0.0230],
         [0.0458],
         [0.0842],
         [0.0940],
         [0.1352],
         [0.2615],
         [0.3624],
         [0.5852],
         [0.6488]],

        [[0.0185],
         [0.1034],
         [0.1136],
         [0.1507],
         [0.1821],
         [0.2764],
         [0.3642],
         [0.4186],
         [0.5001],
         [0.5346]],

        [[0.0101],
         [0.0351],
         [0.1284],
         [0.1398],
         [0.1518],
         [0.1607],
         [0.1656],
         [0.1837],
         [0.2238],
         [0.8958]]])


In [90]:
concat_coor_dis = torch.concat([coor,dis_to_center],dim = 2) # bs * 256 * 6
print(concat_coor_dis.shape)

torch.Size([3, 10, 4])


In [65]:
matrix = torch.tensor([[2,3,1],[3,3,3],[1,1,1],[4,2,1],[2,1,1]],dtype=float)
print(matrix)


tensor([[2., 3., 1.],
        [3., 3., 3.],
        [1., 1., 1.],
        [4., 2., 1.],
        [2., 1., 1.]], dtype=torch.float64)


In [66]:
feature_1 = matrix[:,0]
print(feature_1)

tensor([2., 3., 1., 4., 2.], dtype=torch.float64)


In [67]:
mean = torch.mean(matrix,dim=0)
print(mean)
sd = torch.std_mean(matrix,dim=0)  # out 2 tensor, std and mean
print(sd)

tensor([2.4000, 2.0000, 1.4000], dtype=torch.float64)
(tensor([1.1402, 1.0000, 0.8944], dtype=torch.float64), tensor([2.4000, 2.0000, 1.4000], dtype=torch.float64))


In [68]:
standardize_matrix = matrix
for i in range(3):
    standardize_matrix[:,i] = (standardize_matrix[:,i]- sd[1][i])/sd[0][i]
print(standardize_matrix)

tensor([[-0.3508,  1.0000, -0.4472],
        [ 0.5262,  1.0000,  1.7889],
        [-1.2279, -1.0000, -0.4472],
        [ 1.4033,  0.0000, -0.4472],
        [-0.3508, -1.0000, -0.4472]], dtype=torch.float64)


In [63]:
standardize_matrix = matrix
standardize_matrix = (standardize_matrix-sd[1]) / sd[0]
print(standardize_matrix)
#print(matrix)

tensor([[-2.4126, -1.0000, -2.0652],
        [-1.6434, -1.0000,  0.4348],
        [-3.1819, -3.0000, -2.0652],
        [-0.8742, -2.0000, -2.0652],
        [-2.4126, -3.0000, -2.0652]], dtype=torch.float64)


In [92]:
standardize_matrix_1shot = torch.nn.functional.normalize(matrix,dim=1)
print(standardize_matrix)

tensor([[-0.3508,  1.0000, -0.4472],
        [ 0.5262,  1.0000,  1.7889],
        [-1.2279, -1.0000, -0.4472],
        [ 1.4033,  0.0000, -0.4472],
        [-0.3508, -1.0000, -0.4472]], dtype=torch.float64)


In [75]:
covar_mtrx = torch.zeros(3)
step = 0
for i in range(2):
    for j in range(i+1,3):
        covar = torch.sum((standardize_matrix[:,i] - 0)*(standardize_matrix[:,j] ))
        #print(standardize_matrix[:,2], standardize_matrix[:,0])
        covar_mtrx[step] = covar/(standardize_matrix.size(0)-1)
        step +=1
        print("step",i,j)
print(covar_mtrx)
    

step 0 1
step 0 2
step 1 2
tensor([0.4385, 0.2942, 0.5590])


In [70]:
print(standardize_matrix)
covar_mtrx_1 = torch.cov(standardize_matrix.T)
print(covar_mtrx_1)

tensor([[-0.3508,  1.0000, -0.4472],
        [ 0.5262,  1.0000,  1.7889],
        [-1.2279, -1.0000, -0.4472],
        [ 1.4033,  0.0000, -0.4472],
        [-0.3508, -1.0000, -0.4472]], dtype=torch.float64)
tensor([[1.0000, 0.4385, 0.2942],
        [0.4385, 1.0000, 0.5590],
        [0.2942, 0.5590, 1.0000]], dtype=torch.float64)


In [84]:
a = torch.linalg.svd(covar_mtrx_1)
print(a)


torch.return_types.linalg_svd(
U=tensor([[-0.5144,  0.8124,  0.2746],
        [-0.6318, -0.1426, -0.7619],
        [-0.5798, -0.5654,  0.5866]], dtype=torch.float64),
S=tensor([1.8701, 0.7183, 0.4116], dtype=torch.float64),
Vh=tensor([[-0.5144, -0.6318, -0.5798],
        [ 0.8124, -0.1426, -0.5654],
        [ 0.2746, -0.7619,  0.5866]], dtype=torch.float64))


In [94]:
test = torch.cov(standardize_matrix[0:2].T)
print(test)
test_1 = torch.cov(standardize_matrix[2:5].T)
print(test_1)
within_class = test + test_1

tensor([[0.3846, 0.0000, 0.9806],
        [0.0000, 0.0000, 0.0000],
        [0.9806, 0.0000, 2.5000]], dtype=torch.float64)
tensor([[1.7949, 0.7309, 0.0000],
        [0.7309, 0.3333, 0.0000],
        [0.0000, 0.0000, 0.0000]], dtype=torch.float64)


In [103]:
mean_all = torch.mean(standardize_matrix, dim =0)
print(mean_all.shape)
mean_class_1 = torch.mean(standardize_matrix[0:2],dim = 0)
mean_class_0 = torch.mean(standardize_matrix[2:5],dim = 0)
print(mean_class_1.shape)

between_class = 2*(mean_class_1.reshape(3,1)- mean_all.reshape(3,1))*(mean_class_1.reshape(3,1)- mean_all.reshape(3,1)).T
between_class+= 3*(mean_class_0.reshape(3,1)- mean_all.reshape(3,1))*(mean_class_0.reshape(3,1)- mean_all.reshape(3,1)).T
print(between_class)

torch.Size([3])
torch.Size([3])
tensor([[0.0256, 0.2924, 0.1961],
        [0.2924, 3.3333, 2.2361],
        [0.1961, 2.2361, 1.5000]], dtype=torch.float64)


In [ ]:
# How to calculate between class scatter matrix
# Step 1: compute mean vector of each class - based on feature, so if you have 2 classes and each data has 3 feature -> 2 x 3 mean vector
# Step 2: compute mean for all : num_data * 3
# For each class,temp = num_data_in_class * ( mean_that_class - mean all)*( mean_that_class - mean all).T
# Sum of all class temp will be between class



# How to calculate within class scatter matrix
# Compute covariance matrix for each class, if 2 class -> 2 cova matrix
# Sum these matrix -> within class

In [104]:
tet = torch.tensor([[1,0.4],[0.4,1]])
out = torch.linalg.svd(tet)
print(out)

torch.return_types.linalg_svd(
U=tensor([[-0.7071, -0.7071],
        [-0.7071,  0.7071]]),
S=tensor([1.4000, 0.6000]),
Vh=tensor([[-0.7071, -0.7071],
        [-0.7071,  0.7071]]))


In [12]:
from __future__ import print_function
import numpy as np 
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
np.random.seed(22)

means = [[2, 2], [4, 2]]
cov = [[.3, .2], [.2, .3]]
N = 10
X0 = np.random.multivariate_normal(means[0], cov, N) # class 1
X1 = np.random.multivariate_normal(means[1], cov, N) # class -1 
X = np.concatenate((X0.T, X1.T), axis = 1) # all data 
y = np.concatenate((np.ones((1, N)), -1*np.ones((1, N))), axis = 1) # labels 

In [16]:
print(X0.shape)
print(X1)

(10, 2)
[[3.42746579 0.71254431]
 [4.24760864 2.39846497]
 [3.33595491 1.61731637]
 [3.69420104 1.94273986]
 [4.53897645 2.54957308]
 [3.3071994  0.19362396]
 [4.13924705 2.09561534]
 [4.47383468 2.41269466]
 [4.00512009 1.89290099]
 [4.28205624 1.79675607]]


In [27]:
from cvxopt import matrix, solvers
# build K
V = np.concatenate((X0.T, -X1.T), axis = 1)
print("V=")
print(V.shape)
K = matrix(V.T.dot(V)) # see definition of V, K near eq (8)
print("K-shape",K.size)

p = matrix(-np.ones((2*N, 1))) # all-one vector 
print("p-shape",p.size)
# build A, b, G, h 
G = matrix(-np.eye(2*N)) # for all lambda_n >= 0
print("G-shape",G.size)
h = matrix(np.zeros((2*N, 1)))
print("h-shape",h.size)
A = matrix(y) # the equality constrain is actually y^T lambda = 0
print("A-shape",A.shape)
b = matrix(np.zeros((1, 1))) 
solvers.options['show_progress'] = False
sol = solvers.qp(K, p, G, h, A, b)

l = np.array(sol['x'])
print('lambda = ')
print(l.T)

V=
(2, 20)
K-shape (20, 20)
p-shape (20, 1)
G-shape (20, 20)
h-shape (20, 1)


AttributeError: 'cvxopt.base.matrix' object has no attribute 'shape'

In [4]:
epsilon = 1e-6 # just a small number, greater than 1e-9
S = np.where(l > epsilon)[0]

VS = V[:, S]
XS = X[:, S]
yS = y[:, S]
lS = l[S]
# calculate w and b
w = VS.dot(lS)
b = np.mean(yS.T - w.T.dot(XS))

print('w = ', w.T)
print('b = ', b)

w =  [[-2.00984381  0.64068336]]
b =  4.668560633868099


In [5]:
from sklearn.svm import SVC

y1 = y.reshape((2*N,))
X1 = X.T # each sample is one row
clf = SVC(kernel = 'linear', C = 1e5) # just a big number 

clf.fit(X1, y1) 

w = clf.coef_
b = clf.intercept_
print('w = ', w)
print('b = ', b)

w =  [[-2.00971102  0.64194082]]
b =  [4.66595309]


In [10]:
print(X0)

[[2.37319011 1.71875981]
 [1.51261889 1.40558943]
 [2.4696794  2.02144973]
 [1.78736889 1.29380961]
 [1.81231157 1.56119497]
 [2.03717355 1.93397133]
 [1.53790057 1.87434722]
 [2.29312867 2.76537389]
 [1.38805594 1.86419379]
 [1.57279694 0.90707347]]


In [11]:
print(X1)

[[2.37319011 1.71875981]
 [1.51261889 1.40558943]
 [2.4696794  2.02144973]
 [1.78736889 1.29380961]
 [1.81231157 1.56119497]
 [2.03717355 1.93397133]
 [1.53790057 1.87434722]
 [2.29312867 2.76537389]
 [1.38805594 1.86419379]
 [1.57279694 0.90707347]
 [3.42746579 0.71254431]
 [4.24760864 2.39846497]
 [3.33595491 1.61731637]
 [3.69420104 1.94273986]
 [4.53897645 2.54957308]
 [3.3071994  0.19362396]
 [4.13924705 2.09561534]
 [4.47383468 2.41269466]
 [4.00512009 1.89290099]
 [4.28205624 1.79675607]]
